# Poe

This notebook covers how to build a Poe API Bot server using `ChatPoe` to call on other Poe bots.

In [1]:
import asyncio
from typing import AsyncIterable
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.schema import AIMessage, HumanMessage, SystemMessage, BaseMessage
from sse_starlette.sse import ServerSentEvent

from fastapi_poe import PoeBot
from fastapi_poe.types import QueryRequest, ProtocolMessage
from langchain.chat_models import ChatPoe


system_message = """You are the CatBot. \
You will respond to every message as if you were a cat \
and will always stay in character as a lazy, easily distracted cat. \
Be verbose in your responses so that you get your point across."""


In [2]:
def _protocol_messages_to_langchain_messages(protocol_messages: list[ProtocolMessage]) -> list[BaseMessage]:
    messages: list[BaseMessage] = [SystemMessage(content=system_message)]
    for message in protocol_messages:
        if message.role == "bot":
            messages.append(AIMessage(content=message.content))
        elif message.role == "user":
            messages.append(HumanMessage(content=message.content))
    return messages


class LangChainPoeBot(PoeBot):
    async def get_response(self, query: QueryRequest) -> AsyncIterable[ServerSentEvent]:
        """Stream bot responses to Poe."""
        handler = AsyncIteratorCallbackHandler()
        chat_poe = ChatPoe(
            bot_name="Assistant",
            api_key=query.api_key,
            user_id=query.user_id,
            conversation_id=query.conversation_id,
            message_id=query.message_id,
            version=query.version,
            callbacks=[handler],
        )

        messages = _protocol_messages_to_langchain_messages(query.query)
        asyncio.create_task(chat_poe.agenerate([messages]))
        async for token in handler.aiter():
            yield self.text_event(token)

# Deploying the bot

You can follow the instructions in the [Poe API docs](https://developer.poe.com/api-bots/quick-start) to deploy your bot and register it with Poe.